In [1]:
library(data.table)
library(stringr)

In [50]:
# files
tss_out_file="/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/overlaps/overlaps_hg38_TSS1kb_withput_Sasse2019nascent_MUMERGE_50bp_04-4-24.bed"
count_out_file="/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/overlaps/overlaps_hg38_withput_Sasse2019nascent_MUMERGE_600bp_04-4-24.bed"
closest_file="/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/overlaps/closest_hg38_withput_Sasse2019nascent_MUMERGE_600bp_04-4-24.bed"
og_bid_file="/Users/hoto7260/projects/Resp_Env/Sasse2019/data/beds/Sasse2019nascent_04-4-24_MUMERGE_600bpwin_tfit,dreg.sorted.bed"

# Get the TSS bids

In [3]:
over <- fread(tss_out_file)
over[1:3,]
nrow(over)
length(unique(over$V8))

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>
chr1,17605,17655,chr1:17561-17699-tfit,chr1,16936,17936,MIR6859-1:NR_106918.1,.,-,50
chr1,29231,29281,chr1:28793-29719-dreg,chr1,28870,29870,WASH7P:NR_024540.1,.,-,50
chr1,29231,29281,chr1:28793-29719-dreg,chr1,29273,30273,MIR1302-2HG:XR_007065314.1,.,+,8


[1] 39457

[1] 33217

In [4]:
# Get difference between mu and TSS
over$mu <- as.integer((over$V3+over$V2)/2)
over$tss <- as.integer((over$V6+over$V7)/2)

# get the distance between the two
over$MUDIFF <- abs(over$mu-over$tss)

over[1:3,]

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,mu,tss,MUDIFF
<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
chr1,17605,17655,chr1:17561-17699-tfit,chr1,16936,17936,MIR6859-1:NR_106918.1,.,-,50,17630,17436,194
chr1,29231,29281,chr1:28793-29719-dreg,chr1,28870,29870,WASH7P:NR_024540.1,.,-,50,29256,29370,114
chr1,29231,29281,chr1:28793-29719-dreg,chr1,29273,30273,MIR1302-2HG:XR_007065314.1,.,+,8,29256,29773,517


In [5]:
over$Gene <- str_split_fixed(over$V8, ":", 2)[,1]

trans_with_tss <- unique(over$V8)
length(trans_with_tss)
genes_with_tss <- unique(over$Gene)
length(genes_with_tss)

[1] 33217

[1] 17054

In [6]:
over <- data.frame(over)
over$unique <- seq(1, nrow(over))
hand_annotate = c()
TSS_used = c()
keep <- c()
remove <- c()
for (isoform in trans_with_tss) {
    # get the overlaps
    filt = over[over$V8 == isoform,]
    # get the TSS with the minimum MUDIFF
    tss = filt[filt$MUDIFF == min(filt$MUDIFF),]$V4
    #cat("\n", isoform, tss)
    if (length(tss) != 1) {
        hand_annotate <- c(hand_annotate, isoform)
        cat("\nThe isoform", isoform, "has", length(tss), "TSS that are closest. Hand annotate")
    } else if (tss %in% TSS_used) {
        # if the TSS has already been assigned to a gene
        # check the difference in MUDIFF between the two genes
        filt2 = over[over$V4 == tss,]
        # if the MUDIFF of the two genes are significantly different, only keep one
        if (max(abs(filt2$MUDIFF))-min(abs(filt2$MUDIFF)) > 50) {
            keep <- c(keep, filt2[abs(filt2$MUDIFF) == min(abs(filt2$MUDIFF)),]$unique)
            remove <- c(remove, filt2[abs(filt2$MUDIFF) == max(abs(filt2$MUDIFF)),]$unique)
        } else {
            keep <- c(keep, filt$unique)
        }}else {
            keep <- c(keep, filt[filt$V4 == tss,]$unique)
            }
    TSS_used <- union(TSS_used, tss)
        }


The isoform UVSSA:XM_017008490.1 has 2 TSS that are closest. Hand annotate

In [ ]:
keep[1:4]

In [8]:
length(hand_annotate)
test <- over[over$V8 == "UVSSA:XM_017008490.1",]
test[1:4,]

# none should be kept


[1] 1

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,mu,tss,MUDIFF,Gene,unique
,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>
27779,chr4,1346788,1346838,chr4:1346594-1347032-tfit,chr4,1346563,1347563,UVSSA:XM_017008490.1,.,+,50,1346813,1347063,250,UVSSA,27779
27781,chr4,1347288,1347338,"chr4:1347032-1347594-tfit,dreg",chr4,1346563,1347563,UVSSA:XM_017008490.1,.,+,50,1347313,1347063,250,UVSSA,27781
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA.1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [9]:
setdiff(keep2, keep)
keep <- union(keep, keep2)

numeric(0)

In [13]:
over_filt <- over[over$unique %in% keep,]
over_filt <- over_filt[!over_filt$unique %in% remove,]

trans_with_tss_filt <- unique(over_filt$V8)
cat("\nOriginal # Transcripts captured:", length(trans_with_tss), "and new #:", length(trans_with_tss_filt))

genes_with_tss_filt <- unique(over_filt$Gene)
cat("\nOriginal # Genes captured:", length(genes_with_tss), "and new #:", length(genes_with_tss_filt))

cat("\nOriginal MUDIFF quantiles vs Filtered:")
quantile(over$MUDIFF, probs = seq (0, 1, 0.25), na.rm = FALSE)
quantile(over_filt$MUDIFF, probs = seq (0, 1, 0.25), na.rm = FALSE)


Original # Transcripts captured: 33217 and new #: 26452
Original # Genes captured: 17054 and new #: 15161
Original MUDIFF quantiles vs Filtered

0%  25%  50%  75% 100% 
   0   57  130  286  524

0%  25%  50%  75% 100% 
   0   39   87  176  524

In [16]:
# Now save the TSS bids
colnames(over_filt) <- c("Chr", "Bid_Start", "Bid_Stop", "BidID", "Gene_Chr", "Gene_Start", "Gene_Stop", 
                        "TranscriptID", ".", "Strand", "Length", "mu", "TSS", "MUDIFF", "GeneID", "unique")
over_filt <- over_filt[,c("Chr", "Bid_Start", "Bid_Stop", "BidID", "Gene_Start", "Gene_Stop",
             "TranscriptID", "Strand", "mu", "TSS", "GeneID")]
over_filt[1:2,]

write.table(over_filt, 
           "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/regions/tss_bid_Sasse2019nascent_50bpwin_04-4-24.txt", 
           row.names=FALSE, sep="\t", quote=FALSE)

,Chr,Bid_Start,Bid_Stop,BidID,Gene_Start,Gene_Stop,TranscriptID,Strand,mu,TSS,GeneID
,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>
1,chr1,17605,17655,chr1:17561-17699-tfit,16936,17936,MIR6859-1:NR_106918.1,-,17630,17436,MIR6859-1
2,chr1,29231,29281,chr1:28793-29719-dreg,28870,29870,WASH7P:NR_024540.1,-,29256,29370,WASH7P


# Filter the Bids according to overlaps
I will remove any bidirectionals overlapping positive and negative genes that both have significant counts (above 40)

In [30]:
count_over <- fread(count_out_file)
count_over[1:2,]
dim(count_over)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,14416,15016,"chr1:14651-14781-tfit,dreg",600
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,16608,17208,chr1:16290-17526-tfit,600


[1] 192198     11

In [31]:
# read in the counts
counts <- fread("/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/counts/Sasse2019_str_put_genes.txt")
dim(counts)
counts[1:3,]
colnames(counts)

[1] 83826    12

Geneid,Chr,Start,End,Strand,Length,/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429054.sorted.bam,/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429055.sorted.bam,/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429056.sorted.bam,/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429057.sorted.bam,/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429058.sorted.bam,/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429059.sorted.bam
<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
DDX11L1:NR_046018.2,chr1,12623,14409,+,1787,0,0,0,0,0,0
WASH7P:NR_024540.1,chr1,14361,28620,-,14260,108,86,71,75,155,107
MIR6859-1:NR_106918.1,chr1,17368,17436,-,69,13,12,8,7,3,7


[1] "Geneid"                                                                   
 [2] "Chr"                                                                      
 [3] "Start"                                                                    
 [4] "End"                                                                      
 [5] "Strand"                                                                   
 [6] "Length"                                                                   
 [7] "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429054.sorted.bam"
 [8] "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429055.sorted.bam"
 [9] "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429056.sorted.bam"
[10] "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429057.sorted.bam"
[11] "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429058.sorted.bam"
[12] "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/bams/SRR8429059.sorted.bam"

In [32]:
colnames(counts) <- c("Geneid", "Chr", "Start", "End", "Strand", "Length", 
                     "ETH_R1", "ETH_R2", "TNF_R1", "TNF_R1", "DEX_R1", "DEX_R2")

counts$Geneid <- str_split_fixed(counts$Geneid, ",", 2)[,1]

length(unique(count_over$V4))
length(intersect(counts$Geneid, count_over$V4))
length(setdiff(count_over$V4, counts$Geneid))
setdiff(count_over$V4, counts$Geneid)

[1] 47886

[1] 47886

[1] 0

character(0)

In [41]:
tnf <- data.frame(counts[,7:10])
dex <- data.frame(counts[,7:8])
dex <- cbind(dex, counts[,11:12])
rownames(tnf) <- counts$Geneid
rownames(dex) <- counts$Geneid

tnf <- tnf[rowSums(tnf) < 40,]
dim(tnf)
dex <- dex[rowSums(dex) < 40,]
dim(dex)

[1] 29906     4

[1] 29281     4

In [42]:
low_counts <- intersect(rownames(tnf), rownames(dex))
length(low_counts)
# use that to account for overlaps to consider
count_over_filt <- count_over[!count_over$V4 %in% low_counts]
dim(count_over)
dim(count_over_filt)

[1] 28307

[1] 192198     11

[1] 189536     11

In [39]:
count_over_filt[1:2,]

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,14416,15016,"chr1:14651-14781-tfit,dreg",600
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,16608,17208,chr1:16290-17526-tfit,600


In [43]:
## Split up in positive and negative
count_over_neg = count_over[count_over$V6 == "-", ]
count_over_pos = count_over[count_over$V6 == "+", ]

pos_bids <- unique(count_over_pos$V10)
neg_bids <- unique(count_over_neg$V10)
pos_neg <- intersect(pos_bids, neg_bids)
length(pos_neg)

count_over_neg = count_over_filt[count_over_filt$V6 == "-", ]
count_over_pos = count_over_filt[count_over_filt$V6 == "+", ]

pos_bids <- unique(count_over_pos$V10)
neg_bids <- unique(count_over_neg$V10)
pos_neg_filt <- intersect(pos_bids, neg_bids)
length(pos_neg_filt)

[1] 9622

[1] 8966

In [44]:
# take out the tss in this list
pos_neg_filt_notss <- setdiff(pos_neg_filt, over_filt$BidID)
length(pos_neg_filt_notss)
pos_neg_filt_notss[1:10]

## After checking the first 4, I agree they should be removed. The first is close to a TSS bid but is not the final one as shown below

[1] 4681

[1] "chr1:974890-975162-tfit"   "chr1:1065855-1066199-dreg"
 [3] "chr1:1310369-1312367-tfit" "chr1:1534752-1535196-tfit"
 [5] "chr1:1624842-1625394-tfit" "chr1:2049472-2049788-tfit"
 [7] "chr1:2181463-2181685-tfit" "chr1:2182714-2183506-tfit"
 [9] "chr1:2184087-2186049-tfit" "chr1:2191704-2191968-tfit"

### Now consider those with 5kb downstream of two transcribed genes

In [46]:
closest <- fread(closest_file)

closest[1:2,]
# only keep those with 10kb distance from either end
closest <- closest[abs(closest$V11) < 10000,]
dim(closest)
# only keep those with genes that are being transcribed significantly
closest <- closest[!closest$V4 %in% low_counts]
dim(closest)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>
chr1,11873,14409,DDX11L1:NR_046018.2,.,+,chr1,14416,15016,"chr1:14651-14781-tfit,dreg",8
chr1,11873,14409,DDX11L1:NR_046018.2,.,+,chr1,16608,17208,chr1:16290-17526-tfit,2200


[1] 317635     11

[1] 299703     11

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,14416,15016,"chr1:14651-14781-tfit,dreg",0
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,16608,17208,chr1:16290-17526-tfit,0
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,17330,17930,chr1:17561-17699-tfit,0
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,28956,29556,chr1:28793-29719-dreg,0
chr1,17368,17436,MIR6859-1:NR_106918.1,.,-,chr1,17330,17930,chr1:17561-17699-tfit,0
chr1,17368,17436,MIR6859-1:NR_106918.1,.,-,chr1,16608,17208,chr1:16290-17526-tfit,-161
chr1,17368,17436,MIR6859-1:NR_106918.1,.,-,chr1,14416,15016,"chr1:14651-14781-tfit,dreg",-2353
chr1,120848,175246,LOC124900384:XR_001737579.3,.,-,chr1,139306,139906,chr1:139353-139859-tfit,0
chr1,120848,175246,LOC124900384:XR_001737579.3,.,-,chr1,184730,185330,chr1:184840-185220-dreg,9485


In [47]:
# only keep those that are within 10kb to the 3prime end 
# for positive genes, means V11 <10000 & positive 
closest_filt <- closest[(closest$V6 == "+" & closest$V11 > 0 & closest$V11 < 10001) | closest$V6 == "-",]
nrow(closest_filt)
# for negative genes, means V11 <10000 & negative
closest_filt <- closest_filt[(closest_filt$V6 == "-" & closest_filt$V11 < 0 & closest_filt$V11 > -10001) | closest_filt$V6 == "+",]
nrow(closest_filt)
# now see which bids are this for two colliding genes
closest[1:3,]

[1] 170612

[1] 51073

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,14416,15016,"chr1:14651-14781-tfit,dreg",0
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,16608,17208,chr1:16290-17526-tfit,0
chr1,14361,29370,WASH7P:NR_024540.1,.,-,chr1,17330,17930,chr1:17561-17699-tfit,0


In [48]:
pos_colliding <- closest_filt[closest_filt$V6 == "+",]$V10
neg_colliding <- closest_filt[closest_filt$V6 == "-",]$V10
pos_neg_colliding <- intersect(pos_colliding, neg_colliding)
length(pos_neg_colliding)

[1] 3195

In [49]:
pos_neg_colliding_notss <- setdiff(pos_neg_colliding, over_filt$BidID)
length(pos_neg_colliding_notss)

[1] 2680

### Make a Saf for the bids according to these

In [51]:
# read in the original bids
bids <- fread(og_bid_file)
bids[1:2,]

V1,V2,V3,V4
<chr>,<int>,<int>,<chr>
chr1,14416,15016,"chr1:14651-14781-tfit,dreg"
chr1,16608,17208,chr1:16290-17526-tfit


In [53]:
length(pos_neg_colliding_notss)
length(pos_neg_filt_notss)
length(intersect(pos_neg_colliding_notss, pos_neg_filt_notss))
pos_neg_rem <- union(pos_neg_colliding_notss, pos_neg_filt_notss)
length(pos_neg_rem)
length(intersect(bids$V4, pos_neg_rem))

[1] 2680

[1] 4681

[1] 188

[1] 7173

[1] 7173

In [55]:
bids <- bids[!bids$V4 %in% pos_neg_rem]
dim(bids)
colnames(bids) <- c("Chr", "Start", "End", "GeneID")
bids$Strand <- "+"
bids[1:2,]

[1] 78455     4

Chr,Start,End,GeneID,Strand
<chr>,<int>,<int>,<chr>,<chr>
chr1,14416,15016,"chr1:14651-14781-tfit,dreg",+
chr1,16608,17208,chr1:16290-17526-tfit,+


In [56]:
write.table(bids[,c("GeneID", "Chr", "Start", "End", "Strand")], 
           "/scratch/Users/hoto7260/Resp_Env/Sasse2019/out/regions/Sasse2019_MUMERGE_600bpwin_tfit,dreg_04-4-24_filt.saf", 
           quote=FALSE, sep="\t", row.names=FALSE)